In [1]:
import librosa
import numpy as np
import pandas as pd
from scipy import signal
import noisereduce as nr

import scipy
from scipy.stats import skew, kurtosis

import warnings
warnings.filterwarnings('ignore')

from IPython.display import Audio

In [2]:
class AudioPreprocessor:
    def __init__(self, sample_rate=44100):
        self.sample_rate = sample_rate
        
    def load_audio(self, file_path):
        """Carga un archivo de audio."""
        audio, sr = librosa.load(file_path, sr=self.sample_rate)
        return audio
    
    def normalize_volume(self, audio, target_dBFS=-20):
        """Normaliza el volumen del audio a un nivel objetivo en dBFS."""
        rms = librosa.feature.rms(y=audio)[0]
        mean_rms = np.mean(rms)
        current_dBFS = 20 * np.log10(mean_rms) if mean_rms > 0 else -np.inf
        adjustment = target_dBFS - current_dBFS
        normalized_audio = audio * (10 ** (adjustment / 20))
        # Prevenir clipping
        normalized_audio = np.clip(normalized_audio, -1.0, 1.0)
        return normalized_audio
    
    def remove_silence(self, audio, top_db=30):
        """Elimina silencios al inicio y final del audio."""
        return librosa.effects.trim(audio, top_db=top_db)[0]
    
    def apply_bandpass_filter(self, audio, lowcut=80, highcut=8000):
        """Aplica un filtro paso banda para reducir ruido."""
        nyquist = self.sample_rate // 2
        low = lowcut / nyquist
        high = highcut / nyquist
        b, a = signal.butter(4, [low, high], btype='band')
        return signal.filtfilt(b, a, audio)
    
    def reduce_noise(self, audio, noise_audio=None):
        """Reduce el ruido de un audio utilizando un perfil de ruido."""
        if noise_audio is None:
            noise_audio = audio[:int(len(audio) * 0.1)]
        
        reduced_audio = nr.reduce_noise(y=audio, y_noise=noise_audio, sr=self.sample_rate)
        return reduced_audio
    
    def clip_prevention(self, audio, threshold=0.95):
        """Previene el clipping manteniendo la señal dentro de los límites."""
        max_val = np.max(np.abs(audio))
        if max_val > threshold:
            audio = audio * (threshold / max_val)
        return audio
    
    def process_audio(self, file_path):
        """Aplica toda la cadena de preprocesamiento a un archivo de audio."""
        # Cargar audio
        audio = self.load_audio(file_path)
        
        # Aplicar preprocesamiento
        audio = self.remove_silence(audio)
        audio = self.normalize_volume(audio)
        audio = self.apply_bandpass_filter(audio)
        audio = self.reduce_noise(audio)
        audio = self.clip_prevention(audio)
            
        return audio

In [3]:


class VoiceFeatureExtractor:
    def __init__(self, sample_rate=44100, n_mfcc=13, n_mels=128, frame_length=0.025, 
                 frame_step=0.01, nfilt=26, window='hamming'):
        """
        Inicializa el extractor de características.
        
        Args:
            sample_rate (int): Frecuencia de muestreo
            n_mfcc (int): Número de coeficientes MFCC
            n_mels (int): Número de bandas mel
            frame_length (float): Longitud de la ventana en segundos
            frame_step (float): Paso entre ventanas en segundos
            nfilt (int): Número de filtros mel
            window (str): Tipo de ventana ('hamming', 'hanning', etc.)
        """
        self.sample_rate = sample_rate
        self.n_mfcc = n_mfcc
        self.n_mels = n_mels
        self.frame_length = int(frame_length * sample_rate)
        self.frame_step = int(frame_step * sample_rate)
        self.nfilt = nfilt
        self.window = window
        
    def extract_features(self, audio):
        """
        Extrae todas las características del audio.
        """
        features = {}
        
        # 1. Características temporales
        features.update(self._extract_temporal_features(audio))
        
        # 2. Características espectrales
        features.update(self._extract_spectral_features(audio))
        
        # 3. Características cepstrales
        features.update(self._extract_cepstral_features(audio))
        
        # 4. Características de energía
        features.update(self._extract_energy_features(audio))
        
        # 5. Características rítmicas
        features.update(self._extract_rhythm_features(audio))
        
        return features
    
    def _extract_temporal_features(self, audio):
        """Extrae características del dominio temporal."""
        features = {}
        
        # Zero Crossing Rate
        zcr = librosa.feature.zero_crossing_rate(audio, 
                                               frame_length=self.frame_length, 
                                               hop_length=self.frame_step)[0]
        features.update({
            'zcr_mean': np.mean(zcr),
            'zcr_std': np.std(zcr),
            'zcr_skew': skew(zcr),
            'zcr_kurtosis': kurtosis(zcr),
            'zcr_median': np.median(zcr)
        })
        
        # Amplitud envelope
        envelope = np.abs(scipy.signal.hilbert(audio))
        features.update({
            'envelope_mean': np.mean(envelope),
            'envelope_std': np.std(envelope),
            'envelope_skew': skew(envelope),
            'envelope_kurtosis': kurtosis(envelope)
        })
        
        return features
    
    def _extract_spectral_features(self, audio):
        """Extrae características del dominio espectral."""
        features = {}
        
        # Centroide espectral
        spectral_centroids = librosa.feature.spectral_centroid(y=audio, 
                                                             sr=self.sample_rate,
                                                             n_fft=self.frame_length,
                                                             hop_length=self.frame_step,
                                                             window=self.window)[0]
        features.update({
            'spectral_centroid_mean': np.mean(spectral_centroids),
            'spectral_centroid_std': np.std(spectral_centroids),
            'spectral_centroid_skew': skew(spectral_centroids)
        })
        
        # Rolloff espectral
        spectral_rolloff = librosa.feature.spectral_rolloff(y=audio, 
                                                          sr=self.sample_rate,
                                                          n_fft=self.frame_length,
                                                          hop_length=self.frame_step,
                                                          window=self.window)[0]
        features.update({
            'spectral_rolloff_mean': np.mean(spectral_rolloff),
            'spectral_rolloff_std': np.std(spectral_rolloff)
        })
        
        # Ancho de banda espectral
        spectral_bandwidth = librosa.feature.spectral_bandwidth(y=audio, 
                                                              sr=self.sample_rate,
                                                              n_fft=self.frame_length,
                                                              hop_length=self.frame_step,
                                                              window=self.window)[0]
        features.update({
            'spectral_bandwidth_mean': np.mean(spectral_bandwidth),
            'spectral_bandwidth_std': np.std(spectral_bandwidth)
        })
        
        # Contraste espectral
        spectral_contrast = librosa.feature.spectral_contrast(y=audio, 
                                                            sr=self.sample_rate,
                                                            n_fft=self.frame_length,
                                                            hop_length=self.frame_step)
        features.update({
            'spectral_contrast_mean': np.mean(spectral_contrast),
            'spectral_contrast_std': np.std(spectral_contrast)
        })
        
        # Flatness espectral
        spectral_flatness = librosa.feature.spectral_flatness(y=audio,
                                                            n_fft=self.frame_length,
                                                            hop_length=self.frame_step)[0]
        features.update({
            'spectral_flatness_mean': np.mean(spectral_flatness),
            'spectral_flatness_std': np.std(spectral_flatness)
        })
        
        return features
    
    def _extract_cepstral_features(self, audio):
        """Extrae características cepstrales."""
        features = {}
        
        # MFCC y sus derivadas
        mfccs = librosa.feature.mfcc(y=audio, 
                               sr=self.sample_rate, 
                               n_mfcc=self.n_mfcc,
                               n_fft=self.frame_length,
                               hop_length=self.frame_step,
                               window=self.window,
                               n_mels=self.nfilt) 
    
        # Delta y Delta-Delta
        mfccs_delta = librosa.feature.delta(mfccs)
        mfccs_delta2 = librosa.feature.delta(mfccs, order=2)
        
        # Estadísticas para cada coeficiente MFCC
        for i in range(self.n_mfcc):
            features.update({
                f'mfcc_{i}_mean': np.mean(mfccs[i]),
                f'mfcc_{i}_std': np.std(mfccs[i]),
                f'mfcc_{i}_skew': skew(mfccs[i]),
                f'mfcc_{i}_delta_mean': np.mean(mfccs_delta[i]),
                f'mfcc_{i}_delta_std': np.std(mfccs_delta[i]),
                f'mfcc_{i}_delta2_mean': np.mean(mfccs_delta2[i]),
                f'mfcc_{i}_delta2_std': np.std(mfccs_delta2[i])
            })
        
        return features
    
    def _extract_energy_features(self, audio):
        """Extrae características relacionadas con la energía."""
        features = {}
        
        # RMS Energy
        rms = librosa.feature.rms(y=audio,
                                frame_length=self.frame_length,
                                hop_length=self.frame_step)[0]
        features.update({
            'rms_mean': np.mean(rms),
            'rms_std': np.std(rms),
            'rms_skew': skew(rms),
            'rms_kurtosis': kurtosis(rms)
        })
        
        # Energía por bandas de frecuencia
        mel_spec = librosa.feature.melspectrogram(y=audio, 
                                                sr=self.sample_rate,
                                                n_fft=self.frame_length,
                                                hop_length=self.frame_step,
                                                n_mels=self.n_mels,
                                                window=self.window)
        
        # Dividir en tercios el espectrograma mel para energía por bandas
        band_size = self.n_mels // 3
        for i in range(3):
            band = mel_spec[i*band_size:(i+1)*band_size]
            features.update({
                f'band_{i}_energy_mean': np.mean(np.sum(band, axis=0)),
                f'band_{i}_energy_std': np.std(np.sum(band, axis=0))
            })
        
        return features
    
    def _extract_rhythm_features(self, audio):
        """Extrae características rítmicas."""
        features = {}
        
        # Onset strength
        onset_env = librosa.onset.onset_strength(y=audio, 
                                               sr=self.sample_rate,
                                               hop_length=self.frame_step)
        features.update({
            'onset_strength_mean': np.mean(onset_env),
            'onset_strength_std': np.std(onset_env)
        })
        
        # Tempo y beats
        tempo, _ = librosa.beat.beat_track(y=audio, 
                                         sr=self.sample_rate,
                                         hop_length=self.frame_step)
        features['tempo'] = tempo
        
        return features
    
    def process_audio(self, audio):
        """
        Procesa un audio y extrae todas sus características.
        
        Args:
            audio (numpy.array): Señal de audio preprocesada
            
        Returns:
            dict: Diccionario con todas las características
        """
        if len(audio) > self.sample_rate:
            audio = audio[:self.sample_rate]
        elif len(audio) < self.sample_rate:
            audio = np.pad(audio, (0, self.sample_rate - len(audio)))
        
        return self.extract_features(audio)

In [4]:
def create_feature_dataset(audio_files, preprocessor, feature_extractor):
    all_features = []
    for idx, file_info in enumerate(audio_files):
        # Preprocesar audio
        audio = preprocessor.process_audio(file_info['ruta_archivo'])
        
        # Extraer características
        features = feature_extractor.process_audio(audio)
        
        # Agregar etiquetas
        features['palabra'] = file_info['palabra']
        # features['persona'] = file_info['persona']
        
        all_features.append(features)
        
        if (idx + 1) % 100 == 0:
            print(f"Procesados {idx + 1} archivos de {len(audio_files)}")
            
    return pd.DataFrame(all_features)

In [5]:
# Crear instancias
preprocessor = AudioPreprocessor()
feature_extractor = VoiceFeatureExtractor()

df = pd.read_csv('dataset.csv')

features_df = create_feature_dataset(df.to_dict('records'), preprocessor, feature_extractor)

features_df.to_csv('voice_features.csv', index=False)

Procesados 100 archivos de 4000
Procesados 200 archivos de 4000
Procesados 300 archivos de 4000
Procesados 400 archivos de 4000
Procesados 500 archivos de 4000
Procesados 600 archivos de 4000
Procesados 700 archivos de 4000
Procesados 800 archivos de 4000
Procesados 900 archivos de 4000
Procesados 1000 archivos de 4000
Procesados 1100 archivos de 4000
Procesados 1200 archivos de 4000
Procesados 1300 archivos de 4000
Procesados 1400 archivos de 4000
Procesados 1500 archivos de 4000
Procesados 1600 archivos de 4000
Procesados 1700 archivos de 4000
Procesados 1800 archivos de 4000
Procesados 1900 archivos de 4000
Procesados 2000 archivos de 4000
Procesados 2100 archivos de 4000
Procesados 2200 archivos de 4000
Procesados 2300 archivos de 4000
Procesados 2400 archivos de 4000
Procesados 2500 archivos de 4000
Procesados 2600 archivos de 4000
Procesados 2700 archivos de 4000
Procesados 2800 archivos de 4000
Procesados 2900 archivos de 4000
Procesados 3000 archivos de 4000
Procesados 3100 arc